## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

In [43]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [44]:
import os
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Bayesian_Optimization_Case_Study.ipynb',
 'flight_delays_test.csv',
 'flight_delays_train.csv']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [45]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [46]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [47]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.024     | 1.205     | 5.819     |
| 2         | 8.162     | 1.486     | 6.676     |
| 3         | 8.401     | 1.763     | 6.638     |
| 4         | 9.343     | 2.343     | 7.0       |
| 5         | 9.645     | 3.0       | 6.645     |


Great, now let's print the best parameters and the associated maximized target.

In [48]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 6.645431253115283}


9.645431253115284

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [49]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [50]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [51]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [52]:
train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [53]:
#Inspect train_df
train_df

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [54]:
#Inspect y_train data with Y mapped to 1 and N mapped to 0
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [55]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [56]:
#Concatenate training & testing dfs
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])

In [57]:
#Inspect concatenated training & test df
full_df

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423
...,...,...,...,...,...,...,...,...
99995,c-6,c-5,c-2,852,WN,CRP,HOU,187
99996,c-11,c-24,c-6,1446,UA,ORD,LAS,1515
99997,c-1,c-30,c-2,1509,OO,ORD,SGF,438
99998,c-1,c-5,c-5,804,DL,LGA,ATL,761


In [58]:
#Use feature_eng() function on concatenated df
full_df = feature_eng(full_df)

In [59]:
#Inspect concatenated df after feature engineering
full_df

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,AA,ATL,DFW,732,ATLDFW,0,0,...,1016,8290,11375,18024,1569,0.343660,-0.939094,ATLDFWAA,DFWAA,ATLAA
1,4,20,3,US,PIT,MCO,834,PITMCO,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,PITMCOUS,MCOUS,PITUS
2,9,2,5,XE,RDU,CLE,416,RDUCLE,1,0,...,136,2246,1747,11737,977,-0.835807,-0.549023,RDUCLEXE,CLEXE,RDUXE
3,11,25,6,OO,DEN,MEM,872,DENMEM,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,DENMEMOO,MEMOO,DENOO
4,10,7,6,WN,MDW,OMA,423,MDWOMA,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,MDWOMAWN,OMAWN,MDWWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,5,2,WN,CRP,HOU,187,CRPHOU,1,0,...,16,1513,224,30958,2558,-0.612907,0.790155,CRPHOUWN,HOUWN,CRPWN
99996,11,24,6,UA,ORD,LAS,1515,ORDLAS,0,0,...,773,4920,9823,13311,1065,-0.799685,-0.600420,ORDLASUA,LASUA,ORDUA
99997,1,30,2,OO,ORD,SGF,438,ORDSGF,0,0,...,795,278,9823,15343,1149,-0.690251,-0.723570,ORDSGFOO,SGFOO,ORDOO
99998,1,5,5,DL,LGA,ATL,761,LGAATL,1,0,...,283,11379,3350,14624,1291,-0.509041,0.860742,LGAATLDL,ATLDL,LGADL


In [60]:
#Use .info() on full_df after feature engineering
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199983 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     199983 non-null  int32  
 1   DayofMonth                199983 non-null  uint8  
 2   DayOfWeek                 199983 non-null  uint8  
 3   UniqueCarrier             199983 non-null  object 
 4   Origin                    199983 non-null  object 
 5   Dest                      199983 non-null  object 
 6   Distance                  199983 non-null  int64  
 7   flight                    199983 non-null  object 
 8   begin_of_month            199983 non-null  uint8  
 9   midddle_of_month          199983 non-null  uint8  
 10  end_of_month              199983 non-null  uint8  
 11  hour                      199983 non-null  int32  
 12  morning                   199983 non-null  uint8  
 13  day                       199983 non-null  ui

Apply the earlier defined feature engineering functions to the full dataframe.

In [61]:
#Use label_enc function on specified columns in list below using for-loop on concatenated training & test df
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

In [62]:
#Inspect full_df after label_enc() function
full_df

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1016,8290,11375,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2246,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,5,2,20,71,132,187,1002,1,0,...,16,1513,224,30958,2558,-0.612907,0.790155,1680,766,413
99996,11,24,6,18,213,159,1515,3635,0,0,...,773,4920,9823,13311,1065,-0.799685,-0.600420,6205,955,1331
99997,1,30,2,16,213,260,438,3689,0,0,...,795,278,9823,15343,1149,-0.690251,-0.723570,6331,1719,1330
99998,1,5,5,7,166,19,761,2685,1,0,...,283,11379,3350,14624,1291,-0.509041,0.860742,4650,71,994


In [64]:
#Use .info() on full_df after label_enc
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199983 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     199983 non-null  int32  
 1   DayofMonth                199983 non-null  uint8  
 2   DayOfWeek                 199983 non-null  uint8  
 3   UniqueCarrier             199983 non-null  int32  
 4   Origin                    199983 non-null  int32  
 5   Dest                      199983 non-null  int32  
 6   Distance                  199983 non-null  int64  
 7   flight                    199983 non-null  int32  
 8   begin_of_month            199983 non-null  uint8  
 9   midddle_of_month          199983 non-null  uint8  
 10  end_of_month              199983 non-null  uint8  
 11  hour                      199983 non-null  int32  
 12  morning                   199983 non-null  uint8  
 13  day                       199983 non-null  ui


Split the new full dataframe into X_train and X_test. 

In [65]:
#Select X_train & X_test
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [66]:
#Inspect X_train
X_train

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,...,1016,8290,11375,18024,1569,3.436597e-01,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,...,105,3523,1390,13069,1094,-6.129071e-01,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,...,136,2246,1747,11737,977,-8.358074e-01,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,...,514,1785,6222,15343,1242,-8.849876e-01,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,...,226,687,2571,30958,2674,7.323820e-02,-0.997314,5144,1313,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,4,3,16,259,237,199,4489,1,0,...,305,51,3694,15343,1259,-4.586496e-01,-0.888617,7711,1512,1693
99996,1,18,3,5,98,76,884,1696,0,1,...,337,93,4201,8701,699,-5.090414e-01,0.860742,2890,435,576
99997,1,24,2,14,88,139,1076,1556,0,0,...,356,5769,4315,12005,980,2.613469e-01,-0.965245,2686,820,534
99998,4,27,4,13,82,114,140,1392,0,0,...,677,30,8163,14424,1253,-6.788007e-01,-0.734323,2423,667,500


In [67]:
#Inspect shape of X_train
X_train.shape

(99983, 33)

In [68]:
#Inspect X_test
X_test

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,7,25,3,22,198,222,598,3228,0,0,...,22,5571,221,6029,518,-0.039260,0.999229,5514,1432,1190
1,4,17,2,20,159,132,1235,2523,0,1,...,427,1513,4932,30958,2578,-0.355923,0.934515,4285,766,950
2,12,2,7,13,122,213,577,1955,1,0,...,27,9877,339,14424,1194,-0.133121,0.991100,3307,1340,714
3,3,25,7,20,49,188,377,694,0,0,...,253,571,3029,30958,2570,-0.467930,-0.883766,1183,1145,287
4,6,6,3,18,213,276,258,3697,1,0,...,849,1763,9823,13311,1125,-0.697790,-0.716302,6348,1829,1331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,5,2,20,71,132,187,1002,1,0,...,16,1513,224,30958,2558,-0.612907,0.790155,1680,766,413
99996,11,24,6,18,213,159,1515,3635,0,0,...,773,4920,9823,13311,1065,-0.799685,-0.600420,6205,955,1331
99997,1,30,2,16,213,260,438,3689,0,0,...,795,278,9823,15343,1149,-0.690251,-0.723570,6331,1719,1330
99998,1,5,5,7,166,19,761,2685,1,0,...,283,11379,3350,14624,1291,-0.509041,0.860742,4650,71,994


In [70]:
#Inspect shape of X_test
X_test.shape

(100000, 33)

Create a list of the categorical features.

In [71]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [72]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       callbacks= [lightgbm.early_stopping(100)],
                       stratified=True,
                       nfold=3)
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [73]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)


|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[952]	cv_agg's valid auc: 0.744709 + 0.00209772
| 1         | 0.7447    | 0.008764  | 0.02315   | 7.681     | 2.099e+03 | 1.616e+03 | 1.166e+03 |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[459]	cv_agg's valid auc: 0.733569 + 0.00366396
| 2         | 0.7336    | 0.01207   | 0.009733  | 50.33     | 6.042e+03 | 992.9     | 1.537e+03 |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	cv_agg's valid auc: 0.719241 + 0.00228385
| 3         | 0.7192    | 0.03773   | 0.008057  | 28.21     | 1.21e+03  | 314.3     | 3.452e+03 |
Training until validation scores don't improve for 100 rounds
E

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [74]:
lgbBO.max

{'target': 0.7447085394347273,
 'params': {'lambda_l1': 0.008763956463843259,
  'lambda_l2': 0.023150623175120857,
  'max_depth': 7.681310378368524,
  'min_child_samples': 2098.743878746051,
  'min_data_in_leaf': 1615.8685207733013,
  'num_leaves': 1165.564354430496}}

Review the process at each step by using the '.res[0]' function.

In [75]:
lgbBO.res[0]

{'target': 0.7447085394347273,
 'params': {'lambda_l1': 0.008763956463843259,
  'lambda_l2': 0.023150623175120857,
  'max_depth': 7.681310378368524,
  'min_child_samples': 2098.743878746051,
  'min_data_in_leaf': 1615.8685207733013,
  'num_leaves': 1165.564354430496}}